In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [3]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [4]:
X = np.append(s25,h25,axis=0)

In [5]:
y = np.append(s25_labels,h25_labels,axis=0)

In [6]:
X.shape

(1142, 6250, 19)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'relu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [8]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [10]:
run_experiment(X,y)

Epoch 1/10
29/29 [==============================] - 1s 18ms/step - loss: 34.3931 - accuracy: 0.6046
Epoch 2/10
29/29 [==============================] - 1s 18ms/step - loss: 0.3461 - accuracy: 0.9299
Epoch 3/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0705 - accuracy: 0.9792
Epoch 4/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0124 - accuracy: 0.9956
Epoch 5/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0070 - accuracy: 0.9989
Epoch 6/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 7/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 8/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0037 - accuracy: 0.9989
Epoch 9/10
29/29 [==============================] - 1s 19ms/step - loss: 0.0088 - accuracy: 0.9989
Epoch 10/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0030 - accuracy: 0.9989
 >#1: 89

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'relu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [14]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [15]:
run_experiment_2(X,y)

Epoch 1/10
29/29 [==============================] - 1s 24ms/step - loss: 86.1413 - accuracy: 0.5038
Epoch 2/10
29/29 [==============================] - 1s 24ms/step - loss: 0.8251 - accuracy: 0.6002
Epoch 3/10
29/29 [==============================] - 1s 24ms/step - loss: 0.6332 - accuracy: 0.6156
Epoch 4/10
29/29 [==============================] - 1s 24ms/step - loss: 0.6034 - accuracy: 0.6561
Epoch 5/10
29/29 [==============================] - 1s 24ms/step - loss: 0.5567 - accuracy: 0.6922
Epoch 6/10
29/29 [==============================] - 1s 24ms/step - loss: 0.5596 - accuracy: 0.7043
Epoch 7/10
29/29 [==============================] - 1s 24ms/step - loss: 0.5364 - accuracy: 0.7196
Epoch 8/10
29/29 [==============================] - 1s 24ms/step - loss: 0.5088 - accuracy: 0.7722
Epoch 9/10
29/29 [==============================] - 1s 24ms/step - loss: 0.4821 - accuracy: 0.7831
Epoch 10/10
29/29 [==============================] - 1s 23ms/step - loss: 0.5109 - accuracy: 0.7514
 >#1: 75

In [18]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'relu' ))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'relu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [19]:
def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_3(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [20]:
run_experiment_3(X,y)

Epoch 1/10
29/29 [==============================] - 1s 18ms/step - loss: 21.7121 - accuracy: 0.5794
Epoch 2/10
29/29 [==============================] - 1s 18ms/step - loss: 0.7924 - accuracy: 0.7514
Epoch 3/10
29/29 [==============================] - 1s 18ms/step - loss: 0.3583 - accuracy: 0.8357
Epoch 4/10
29/29 [==============================] - 1s 18ms/step - loss: 0.2280 - accuracy: 0.9288
Epoch 5/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0932 - accuracy: 0.9748
Epoch 6/10
29/29 [==============================] - 1s 18ms/step - loss: 0.1797 - accuracy: 0.9825
Epoch 7/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0773 - accuracy: 0.9737
Epoch 8/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0434 - accuracy: 0.9858
Epoch 9/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0347 - accuracy: 0.9901
Epoch 10/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0342 - accuracy: 0.9890
 >#1: 94